# Data Wrangling Task

**The Ultimate Customer Segmentation Analysis**

The below task involves a series of steps taken to analyze customer data, clean it, extract insights, and generate a report using Python.

Let's begin!!!

Import necessary packages

In [1]:
import pandas as pd # For data frame manipulation
import numpy as np # For handling arithmetic calculations
from datetime import datetime # For handling date/time data

## Step 1: Load the dataset

### Instructions
- Download the CSV file named `mock_data.csv`.
- Read the file into a Pandas DataFrame,but make sure only the first 75% of the rows are loaded.


### Solution
The `mock_data.csv` contains 1000. Therefore we can only load the first 750

In [2]:
df = pd.read_csv("task 1/MOCK_DATA (3).csv") # Read the .csv
to_load = int(0.75*df.shape[0]) # Find the number of rows to load
df = df[:to_load] # Load only the first 75%

df.info() # Confirm only 75% was loaded

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Customer ID      750 non-null    int64 
 1   Country          750 non-null    object
 2   Total Purchases  750 non-null    int64 
 3   Sign-up Date     750 non-null    object
dtypes: int64(2), object(2)
memory usage: 23.6+ KB


## Step 2: Data Cleaning

### Instructions
- Remove any duplicate rows but only if they appear in consecutive order (Hint: You might need a specific Pandas function).
- Convert the "Sign-up Date" column to a proper datetime format, but replace all entries from the year 2020 with NaN.
- Drop all rows where the column "Total Purchases" is less than 5 but only if the customer is from Canada.



### Solution

Confirm if there are any duplicate rows to begin with.

In [3]:
df.duplicated().sum()

np.int64(0)

There are none. However, we shall write code to solve this task.

In [4]:
'''
To remove consecutive duplicates only
'''
df = df[~df.duplicated(keep="first").mask(df.index.to_series().diff() != 1, False)]

'''
To convert the Sign-up Date column to proper date/time format
'''
df["Sign-up Date"] = pd.to_datetime(df["Sign-up Date"])

'''
To replace all 2020 entries with Nan
'''
df.loc[df["Sign-up Date"].dt.year == 2020, "Sign-up Date"] = np.nan

'''
To drop all rows where the column "Total Purchases" is less than 5 but only if the customer is from Canada
'''
df = df.drop(df[(df["Total Purchases"] < 5) & (df["Country"] == "Canada")].index)

df.info() # Confirm the changes were made

<class 'pandas.core.frame.DataFrame'>
Index: 748 entries, 0 to 749
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Customer ID      748 non-null    int64         
 1   Country          748 non-null    object        
 2   Total Purchases  748 non-null    int64         
 3   Sign-up Date     660 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 29.2+ KB


## Step 3: Feature Engineering

### Instructions
Create a new column called `Loyalty Score` based on the following conditions:
- If Total Purchases > 20, assign a score of 3.
- If Total Purchases is between 10 and 20, assign a score of 2.
- If Total Purchases < 10, assign a score of 1, except if the customer has been a member for more than 3 years (use 'Sign-up Date')—then assign a score of 2 instead.

### Solution
We will create a function that can easily perform this task with multiple conditions using `if`, `elif` and `else` statements

In [5]:
def calculate_loyalty_score(row): # Function to calculate loyalty score
    """
    Returns a loyalty score based on total purchases and membership duration.

    Parameters:
    row (pd.Series): Customer data with "Total Purchases" and "Sign-up Date".

    Returns:
    int: Loyalty score (1, 2, or 3).
    """
    membership_years = pd.Timestamp.now().year - row["Sign-up Date"].year

    if row["Total Purchases"] > 20:
        return 3
    elif 10 <= row["Total Purchases"] <= 20:
        return 2
    elif row["Total Purchases"] < 10:
        return 2 if membership_years > 3 else 1

df["Loyalty Score"] = df.apply(calculate_loyalty_score, axis=1) # Apply function

df.info() # Confirm Loyalty Score was created

<class 'pandas.core.frame.DataFrame'>
Index: 748 entries, 0 to 749
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Customer ID      748 non-null    int64         
 1   Country          748 non-null    object        
 2   Total Purchases  748 non-null    int64         
 3   Sign-up Date     660 non-null    datetime64[ns]
 4   Loyalty Score    748 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 35.1+ KB


## Step 4: Data Aggregation & Filtering

### Instructions
Group the data by "Country" and calculate:
- The average and maximum "Total Purchases."
- The most common Loyalty Score for each country, except for Mexico (which should be excluded entirely from this analysis for no clear reason).


### Solution
By first filtering out all the Mexico entries and using `groupby` and `agg` to obtain our summarised statistics.

In [6]:
df_no_mx = df[df["Country"] != "Mexico"] # Filter out Mexico

aggregate_df = df_no_mx.groupby("Country").agg( # Calculate the following:
    Avg_Total_Purchases=("Total Purchases", "mean"), # Average Total Purchases
    Max_Total_Purchases=("Total Purchases", "max"), # Maximum Total Purchases
    Mode_Loyalty_Score=("Loyalty Score", lambda x: x.mode().mean())) # Most common Loyalty Scores and using the averages for multimodal countries

aggregate_df # Display the statistics

,Avg_Total_Purchases,Max_Total_Purchases,Mode_Loyalty_Score
Country,,,
Afghanistan,8.000000,20,2.0
Albania,20.500000,38,2.5
Andorra,11.000000,11,2.0
Angola,15.000000,29,2.5
Argentina,24.692308,45,3.0
...,...,...,...
Vanuatu,22.000000,22,3.0
Venezuela,25.333333,43,3.0
Vietnam,24.333333,47,3.0


## Step 5: Data Export & Report Generation


### Instructions
- Save the final processed DataFrame as a CSV, but:
  - The filename must follow the format: "processed_data_YYYY_MM_DD.csv", where YYYY-MM-DD is today’s date but written in reverse order (DD-MM-YYYY).
  - Ensure that only the columns `["Customer ID", "Loyalty Score", "Country", "Total Purchases"]` are saved in the final file.
- Print the first 10 rows of the processed DataFrame, but shuffle them before displaying (do NOT sort them).

### Solution

In [7]:
today_date = datetime.today().strftime("%Y-%m-%d")
file_name = "processed_data_"+today_date+".csv" # Create the filename


df[["Customer ID", "Loyalty Score", "Country", "Total Purchases"]].to_csv(file_name, index=False) # Save only the specified columns


print(df.head(10).sample(10)) # Print the first 10 rows shuffled

   Customer ID                Country  Total Purchases        Sign-up Date  \
5         1033              Indonesia               20 2016-05-26 22:29:44   
0         1015  Palestinian Territory               12 2022-04-03 21:42:29   
6         1017               Paraguay               14 2022-09-08 16:30:56   
4         1024            Philippines               16 2016-12-13 03:44:04   
8         1029              Argentina               14                 NaT   
2         1025                Nigeria               10 2023-02-20 19:06:14   
1         1029                  China               47 2016-01-08 03:13:12   
9         1014                  China                3                 NaT   
7         1035               Honduras               15 2021-04-14 18:55:03   
3         1006         United Kingdom               48 2022-05-26 12:46:35   

   Loyalty Score  
5              2  
0              2  
6              2  
4              2  
8              2  
2              2  
1       

## Analysis and Findings

Analysis of the dataset and aggregates  was done in the [Findings.ipynb](https://github.com/Data-Epic/data-wrangling-chidimma-ijoma/blob/feature-task/task%201/Findings.ipynb)

Here are the findings:

1. Summary Statistics of the following
  1. Total Purchases
    - Mean : 23.89
    - Median : 24.00
    - Standard deviation : 13.90
  2. Loyalty Score
    - Mean : 2.49
    - Median : 3.00
    - Standard deviation : 0.63
2. The earliest customer sign-up was on 18th March, 2015 and the latest was on 29th February, 2024
3. Australia has the highest Average Total Purchases (47 per customer) while Sierra Leone has the lowest (2 per customer). The distribution of other countries in in the chart in [Findings.md](https://github.com/Data-Epic/data-wrangling-chidimma-ijoma/blob/feature-task/task%201/Findings.md)
4. Japan, China, Indonesia, Russia, Poland, United Kingdom have the highest maximum Total Purchases (48 per customer) while Sierra Leone has the lowest (2 per customer). The distribution of other countries in in the chart in [Findings.md](https://github.com/Data-Epic/data-wrangling-chidimma-ijoma/blob/feature-task/task%201/Findings.md)
5. There is a positive correlation between the Average Total Purchases and the Loyalty Scores as shown in the chart in [Findings.md](https://github.com/Data-Epic/data-wrangling-chidimma-ijoma/blob/feature-task/task%201/Findings.md)
6. China has the highest number of Purchases (151). The distribution of other countries in in the chart in [Findings.md](https://github.com/Data-Epic/data-wrangling-chidimma-ijoma/blob/feature-task/task%201/Findings.md)